In [1]:
# import eikon as ek
import numpy as np
import pickle
import pandas as pd
# import cufflinks as cf
# import configparser as cp

## Connecting to Eikon Data API

This code sets the app_id to connect to the Eikon Data API Proxy which needs to be running locally. It requires the previously created text file eikon.cfg to be in the current working directory.

In [ ]:
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location

In [ ]:
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated

****

## Get a List of RICs

In [11]:
# temporarily, we download data for all RICs we have
# late on, we will remove some of those RICs
# for that purpose, we need to read RICs from three csv files:
# that is 1.banktupt_listv2.csv and 2.*_list.csv files

# set the path to xlsx file
path_xlsx = 'D:\\studyproject\\bankruptcy\\data\\Eikon\\' + \
       'Identifiers_Mapping\\1.CUSIP-to-ISIN\\' # for win
# path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Eikon/' + \
#       'Identifiers_Mapping/1.CUSIP-to-ISIN/' # for mac

# set the path to csv files
path_csv = 'D:\\studyproject\\bankruptcy\\data\\Eikon\\' + \
       'Identifiers_Mapping\\2.ISIN-to-RIC\\' # for win
# path = '/Users/user/Documents/Bankruptcy/bankruptcy/data/Eikon/' + \
#       'Identifiers_Mapping/2.ISIN-to-RIC/' # for mac



# read xlsx bankrupt file, downloaded from Google Drive
# after being manually improved
bankrupt_m = pd.read_excel(path_xlsx + '1.bankrupt_list_v2.xlsx', sheet_name='1.bankrupt_list',
                           dtype=object)

# read final csv files
bankrupt = pd.read_csv(path_csv + '2.bankrupt_list.csv', dtype=object)
healthy = pd.read_csv(path_csv + '2.healthy_list.csv', dtype=object)

print(bankrupt_m.count())
bankrupt_m.head()

Identifier            112
Company               112
Data Deletion Date    112
Deletion Reason       112
Ticker                112
CUSIP                 112
CIK                   100
ISIN                  112
ISINc                 112
RIC                   101
RICc                  112
Country               112
dtype: int64


,Identifier,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc,RIC,RICc,Country
0,001367,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750,US0231842032,.,AMBEQ.OB^H12,.,US
1,002033,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779,US3036981047,.,FCHDQ.PK^K11,.,US
2,004049,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806,US21036U2069,.,CNST.OQ^A11,.,US
3,004352,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878,US2926591098,.,ENERQ.PK^I12,.,US
4,004768,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132,US3390991038,.,FLTWQ.PK^H10,.,US


In [12]:
print(bankrupt.count())
bankrupt.head()

Company               105
Data Deletion Date    105
Deletion Reason       105
Ticker                105
CUSIP                 105
CIK                    93
ISIN                  105
ISINc                 105
RIC                    57
RIC_ek                  6
RICc                  105
Country               105
dtype: int64


,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc,RIC,RIC_ek,RICc,Country
0,Amber Resources Company of C,08/31/2012,2.0,3AMBE,023184203,0000276750,US0231842032,0,NaN,NaN,0,US
1,Fairchild Corp. (The),11/01/2011,2.0,FCHDQ,303698104,0000009779,US3036981047,0,FCHDQ.PK^K11,NaN,1,US
2,Constar International Inc,06/01/2011,2.0,CNSTQ,21036U206,0000029806,US21036U2069,0,NaN,NaN,0,US
3,Energy Conversion Devices In,09/04/2012,2.0,ENERQ,292659109,0000032878,US2926591098,0,ENERQ.PK^I12,NaN,1,US
4,Fleetwood Enterprises Inc.,08/23/2010,2.0,FLTWQ,339099103,0000314132,US3390991038,0,FLTWQ.PK^H10,NaN,1,US


In [13]:
print(healthy.count())
healthy.head()

Company               19158
Data Deletion Date    19158
Deletion Reason        7780
Ticker                19154
CUSIP                 19158
CIK                   13462
ISIN                  19158
ISINc                 19158
RIC                    9945
RIC_ek                 8551
RICc                  19158
Country               18573
dtype: int64


,Company,Data Deletion Date,Deletion Reason,Ticker,CUSIP,CIK,ISIN,ISINc,RIC,RIC_ek,RICc,Country
0,AAR Corp,.,NaN,AIR,000361105,0000001750,US0003611052,0,AIR,AIR,0,US
1,ADC Telecommunications Inc.,12/10/2010,1.0,ADCT,000886309,0000061478,US0008863096,0,NaN,NaN,0,US
2,AFP Imaging Corp,09/15/2014,7.0,IWKS,001058205,0000319126,US0010582056,0,IWKS.PK,IWKS.PK,1,US
3,Alpharma Inc.,12/31/2008,1.0,ALO.2,020813101,0000730469,US0208131013,0,NaN,NaN,0,US
4,American Airlines Group Inc,.,NaN,AAL,02376R102,0000006201,US02376R1023,0,AAL.O - AAL.Z,AAL.O,3,US


In [14]:
# read a dataseries and convert it to a list
def series_to_list(ds):
    '''
    this function takes a dataseries, breaks the double-value
    rows, and returns a list.
    '''
    # drop NANs
    ds.dropna(inplace=True)
    
    the_list = []
    # iterate over series rows
    for index, value in ds.items():
        if ' - ' in value:
            the_list.append(value[:value.find(' - ')])
            the_list.append(value[value.find(' - ') + 3:])
        else:
            the_list.append(value)
    
    return the_list

In [23]:
# actually extract, and merge the lists together
bankrupt_RIC = list(set(series_to_list(bankrupt_m.RIC) + series_to_list(bankrupt.RIC)
                        + series_to_list(bankrupt.RIC_ek)))

healthy_RIC = list(set(series_to_list(healthy.RIC + healthy.RIC_ek)))

print(len(bankrupt_RIC), 'RICs for bankrupt companies, and')
print(len(healthy_RIC), 'RICs for healthy companies.')

101 RICs for bankrupt companies, and
8596 RICs for healthy companies.


In [ ]:
# ============================================================================
# 5. write the code for downloading financial ratios and timeseries for both
# ============================================================================

## And Finally, Download the Data

### Historical Data

In [ ]:
# create a folder for saving data
if not os.path.exists('bankrup_data'):
    os.makedirs('bankrup_data')
    
if not os.path.exists('healthy_data'):
    os.makedirs('healthy_data')

In [ ]:
# for bankrupt copmanies
for ric in bankrupt_RIC:
    try:
        
        df = ek.get_timeseries(ric,  # the RIC for the company
                             fields='*',  # all fields
                             start_date='2000-01-01',  # start date
                             end_date='2019-08-19')  # end date
    except:
        pass

    df.to_csv('10162019/bankrupt_timeseries/{}.csv'.format(ric))

In [ ]:
fields=['CF_NETCHNG', 'CF_ASK', 'CF_BID', 'CF_LAST', 'CF_CLOSE', 'CF_TICK', 'CF_CURR', 'CF_SOURCE', 'CF_HIGH',
        'CF_LOW', 'CF_VOLUME', 'CF_EXCHNG', 'CF_LOTSIZE', 'CF_NAME', 'CF_TIME', 'CF_DATE',
        'TR.CashAndSTInvestments(Period=FY0)', 'TR.LTDebtToTtlCapitalPct(Period=FY0)',
        'TR.RevenuePerShare(Period=FY0)', 'TR.EBITDA(Period=FY0)', 'TR.EBIT(Period=FY0)',
        'TR.TotalCapital(Period=FY0)', 'TR.TangibleBVPSTotalEquity(Period=FY0)',
        'TR.BVPSTotalEquity(Period=FY0)', 'TR.BookValuePerShare(Period=FY0)',
        'TR.IssuanceRetirementOfDebt(Period=FY0)', 'TR.LTDebtNet(Period=FY0)', 'TR.STDebtNet(Period=FY0)',
        'TR.IssuanceRetirementOfStock(Period=FY0)', 'TR.PreferredStockNet(Period=FY0)',
        'TR.CommonStockNet(Period=FY0)', 'TR.TotalCashDividendsPaid(Period=FY0)']



In [ ]:
bankrupt_data, err = ek.get_data(bankrupt_RIC, fields=fields)
bankrupt_data.to_csv('10162019/bankrupt_data/bankrupt_data.csv')

****

In [ ]:

help(ek.get_data)

## Retrieving Historical Data

As a test, consider the retrieval of historical end-of-day data for the Apple, Inc. stock.

In [ ]:
testAAPLdata = ek.get_timeseries('AAPL.O',  # the RIC for Apple, Inc.
                         fields='*',  # all fields
                         start_date='2000-01-01',  # start date
                         end_date='2019-08-19')  # end date

In [ ]:
testAAPLdata.tail()  # final five rows

In [ ]:
testAAPLdata.head()

In [ ]:
testAAPLdata.to_csv('testDataAPPL.csv')

## Test get_data Function

In [ ]:
data_grid, err = ek.get_data(['IBM', 'GOOG.O', 'MSFT.O'],
                             ['TR.TotalReturnYTD', 'TR.WACCBeta', 'YRHIGH', 'YRLOW'])

In [ ]:
data_grid

In [ ]:
# read the list of bankrupt companies
bankrupt = pd.read_csv('bankrupt.csv')
bankruptList = bankrupt['Ticker'].tolist()


## Test the bankruptcy list companies data availability on eikon

In [ ]:
testBR, err = ek.get_data(bankruptList, ['TR.TotalReturnYTD', 'TR.WACCBeta', 'YRHIGH', 'YRLOW'])

In [ ]:
testBR

## Test historical data availability for bankrupt data

In [ ]:
errors = 0
for company in bankruptList:
    try:
        testdata = ek.get_timeseries(company,  # the RIC for Apple, Inc.
                             fields='*',  # all fields
                             start_date='2000-01-01',  # start date
                             end_date='2019-08-19')  # end date
    except:
            errors += 1


print(errors)


In [ ]:
testdata = ek.get_timeseries('ARA.Z',  # the RIC for Apple, Inc.
                             fields='*',  # all fields
                             start_date='2000-01-01',  # start date
                             end_date='2019-08-19')  # end date

In [ ]:
testdata

In [ ]:
test1 = ek.get_timeseries('AMAT',  # the RIC for Apple, Inc.
                             fields='*',  # all fields
                             start_date='2000-01-01',  # start date
                             end_date='2019-08-19')  # end date

In [ ]:
test1.head()

## Retrieving Historical Data

As a test, consider the retrieval of historical end-of-day data for the Apple, Inc. stock.

In [ ]:
testAAPLdata = ek.get_timeseries('AAPL.O',  # the RIC for Apple, Inc.
                         fields='*',  # all fields
                         start_date='2000-01-01',  # start date
                         end_date='2019-08-19')  # end date

In [ ]:
testAAPLdata.tail()  # final five rows

In [ ]:
testAAPLdata.head()

In [ ]:
testAAPLdata.to_csv('testDataAPPL.csv')

## Test get_data Function

In [ ]:
data_grid, err = ek.get_data(['IBM', 'GOOG.O', 'MSFT.O'],
                             ['TR.TotalReturnYTD', 'TR.WACCBeta', 'YRHIGH', 'YRLOW'])

In [ ]:
data_grid